## Run SimSS simulations and plot the results
Version 0.1
(c) Vincent M. Le Corre (https://github.com/VMLC-PV)

In [1]:
# Package import
# %matplotlib inline
# %matplotlib widget
# %matplotlib notebook
# comment the above line if you don't want interactive plots
import os,platform,warnings,time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# Import homemade package by VLC
from SIMsalabim_utils.GetInputPar import * # Get input parameters
from SIMsalabim_utils.RunSim import * # Run simulation
from SIMsalabim_utils.ZimT_plots import * # Plotting functions
from SIMsalabim_utils.CleanFolder import clean_up_output, clean_file_type # Cleaning functions
from SIMsalabim_utils import plot_settings_screen # Plot settings
from SIMsalabim_utils.aux_func import * # Auxiliary functions
## General Inputs
warnings.filterwarnings("ignore")           # Don't show warnings
system = platform.system()                  # Operating system
max_jobs = os.cpu_count()-2                 # Max number of parallel simulations (for number of CPU use: os.cpu_count() )
do_multiprocessing = True                      # Use multiprocessing
if system == 'Windows':                     # cannot easily do multiprocessing in Windows
        max_jobs = 1
        do_multiprocessing = False
        try:                                # kill all running jobs to avoid conflicts
            os.system('taskkill.exe /F /IM simss.exe')
        except:
            pass

# Path to SIMsalabim
path2simu = os.path.join(os.getcwd() ,'SIMsalabim','zimt')
path2devfile = os.path.join(os.getcwd() ,'Example_Data','Perovskite_examples','trPL','device_parameters.txt')

ModuleNotFoundError: No module named 'SIMsalabim_utils'

In [ ]:
from SIMsalabim_utils.tVG_gen import *
def zimt_trPL_Gauss_pulse(tmin,tmax,Gen_pulse,G0,tpulse,width_pulse = 2e-9,tstepmin=1e-10,Gres=1e0,time_exp =False,steps=100,tVG_name='tVG.txt'):
    """Make tVG file for transient photocurrent (TPC) experiment with a gaussian pulse for the light excitation.

    Parameters
    ----------
    tmin : float
        first time step after 0 (unit: s)
    tmax : float
        final time step (unit: s)
    Gen_pulse : float
        Total number of carrier generated by the gaussian pulse (unit: m^-3)
    G0 : float
        background generation rate (i.e. light intensity) (unit: m^-3 s^-1)
    tpulse : float
        middle of the gaussian pulse (unit: s)
    width_pulse : float, optional
        width of the light pulse (unit: s), by default 2e-9
    time_exp : bool, optional
        if True chose exponential time step else keep time step linear, by default False
    steps : int, optional
        if time_exp = True number of exponential time step after voltage switch, by default 100
    tVG_name : str, optional
        tVG_file name, by default 'tVG.txt'
    """    

    # Define laser pulse by a Gaussian and always keep tstep to tstepmin for during the pulse to ensure that the amount of photon is consistent when changing tstep
    pulse_stop = tpulse + width_pulse*10
    t = np.arange(tmin,pulse_stop,tstepmin)
    t=np.insert(t,0,0)
    V,G = [],[]

    for i in t:
        V.append(0)   
    G = gaussian_pulse(t,tpulse,width_pulse,Gen_pulse)
    G[0] = 0 #set G = G0 at t = 0

    for i in range(len(G)):
        if G[i] < Gres:
            G[i] = 0
    
    if Gen_pulse > 0:
        # ensure that the total number of generated charges is equal to Gen
        int_G  = integrate.cumtrapz(G, t, initial=0)
        G = G*Gen_pulse/int_G[-1]
    
    if G0 > 0:
        for i in range(len(G)):
            G[i] = G[i] + G0
            

    if time_exp == True:
        t1 = np.geomspace(pulse_stop,tmax,steps)
    else :
        t1 = np.linspace(pulse_stop, tmax, steps)

    for i in t1:
        G=np.append(G,G0)
        V.append(0)
    
    t = np.append(t,t1)

    for i in range(len(G)): #set G = 0 when G is too small (for stability in ZimT)
        if G[i] < G0:
            G[i] = G0
    
    
    

    tVG = pds.DataFrame(np.stack([t,np.asarray(V),np.asarray(G)]).T,columns=['t','Vext','Gehp'])

    tVG.to_csv(tVG_name,sep=' ',index=False,float_format='%.3e')

In [ ]:
# test 
zimt_trPL_Gauss_pulse(1e-10, 1e-6, 1e20, 3e27, 10e-9)

df = pd.read_csv('tVG.txt', delim_whitespace=True)

# plt.semilogy(df['t'],df['Gehp'])
plt.plot(df['t'],df['Gehp'])
plt.xlim(0,100e-9)


In [ ]:
Gens = [1e22] # Generation rate (m^-3 s^-1)
G0s = [3e27] # Background generation rate (m^-3 s^-1)
## Prepare strings to run
# Fixed string
fixed_str = str(path2devfile) +' -kdirect 5e-17 -Bulk_tr 1e19 -NP 100 -nu_int_LTL 1e-10 -nu_int_RTL 1e-10 -tolDens 1e-10 -minAcc 0.1 -maxAcc 0.1' # add any fixed string to the simulation command

# Initialize 
code_name_lst,str_lst,path_lst,tj_lst,tVG_lst = [],[],[],[],[]
idx = 0
start = time.time()

# Figures control
size_fig = (16, 12)
num_fig_tjs= 0
colors = cm.viridis((np.linspace(0,1,max(len(Gens),4)+1)) ) # Color range for plotting
f_tjs = plt.figure(num_fig_tjs,figsize=size_fig)

run_simu = True
verbose = True
if run_simu:           
    # Generate tVG files and str_lst for light pulse simulation
    for Gen in Gens:
        for G0 in G0s:
            zimt_trPL_Gauss_pulse(1e-10, 1e-5, Gen, G0, 10e-9,steps=1000,time_exp = True, tVG_name=os.path.join(path2simu,'tVG_TrPL_G_{:.1e}_G0_{:.2e}.txt'.format(Gen,G0)))
            str_lst.append(fixed_str+' -tVG_file tVG_TrPL_G_{:.1e}_G0_{:.2e}.txt -tj_file tj_TrPL_G_{:.1e}_G0_{:.2e}.dat'.format(Gen,G0,Gen,G0))
            code_name_lst.append('zimt')
            path_lst.append(path2simu)
            tVG_lst.append('tVG_TrPL_G_{:.1e}_G0_{:.2e}.txt'.format(Gen,G0))
            tj_lst.append('tj_TrPL_G_{:.1e}_G0_{:.2e}.dat'.format(Gen,G0))
    

    # Run ZimT
    # if do_multiprocessing:
    #     run_multiprocess_simu(run_code,code_name_lst,path_lst,str_lst,max_jobs)
    # else:
    for i in range(len(str_lst)):
        run_code(code_name_lst[i],path_lst[i],str_lst[i],show_term_output=True,verbose=verbose)
        
print('Calculation time {:.2f} s'.format(time.time() - start)) # Time in seconds


In [ ]:
plt.figure(0)
for G0 in G0s:
    idx = 0
    for Gen in Gens:
        data_tj = pd.read_csv(os.path.join(path2simu,'tj_TrPL_G_{:.1e}_G0_{:.2e}.dat'.format(Gen,G0)),delim_whitespace=True)
        plt.loglog(data_tj['t'],data_tj['Jdir']/max(data_tj['Jdir']),label=sci_notation(Gen/1e6,sig_fig=1),color=colors[idx])
        # zimt_tj_plot(idx,data_tj,y=['Jdir'],labels=sci_notation(Gen/1e6,sig_fig=1),colors=colors[idx],plot_type=3,save_yes=False,legend=False,pic_save_name = os.path.join(path2simu,'transient.jpg'))
        idx += 1


In [ ]:
from scipy import integrate,constants

k = constants.value('Boltzmann constant')
q = constants.value('elementary charge')
c = constants.value('speed of light in vacuum')
h = constants.value('Planck constant')
# print(h_SI,q,c,kb_SI,T)

def calc_J0rad(Eg,T):
    Es = np.arange(Eg, 4.401, 0.002)
    # Es = np.arange(0.32, 4.401, 0.002)
    EQE_EL = 1
    phi = 2 * np.pi * (((Es*q)**2) * q / ((h**3) * (c**2)) / (
                           np.exp(Es*q / (k*T)) - 1))
    
    fluxcumm = integrate.cumtrapz(phi[::-1], Es[::-1], initial=0)
    fluxaboveE = fluxcumm[::-1] * -1
    J0 = q * fluxaboveE * 0.1/ EQE_EL
    J0[-1] =- np.nan

    return J0[0]





In [ ]:
ParfileDic = ReadParameterFile(path2devfile)
T = float(ParfileDic['T'])
Eg = abs(float(ParfileDic['CB']) - float(ParfileDic['VB']))
J0rad = calc_J0rad(Eg,T)

plt.figure(1)
for G0 in G0s:
    idx = 0
    for Gen in Gens:
       
        data_tj = pd.read_csv(os.path.join(path2simu,'tj_TrPL_G_{:.1e}_G0_{:.2e}.dat'.format(Gen,G0)),delim_whitespace=True)

        # calculate J0rad
        data_tj['QFLS'] = (k*T/q)*np.log(data_tj['Jdir']/J0rad)
        plt.semilogx(data_tj['t']-10e-9,data_tj['QFLS'],color=colors[idx],label=sci_notation(Gen/1e6,sig_fig=1))

        idx += 1
plt.legend()

In [ ]:
# Clean output files from simulation folders
Do_Cleaning = False # Careful, this will delete all files in the folder
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPars',path2simu)
    clean_up_output('log',path2simu)
    clean_up_output('Str4Parallel',path2simu)
    clean_up_output('joblog',path2simu)
    clean_file_type('.jpg',path2simu) # clean all jpg files
    clean_file_type('.png',path2simu) # clean all png files
    